In [1]:
import pandas as pd
import numpy as np
import chromadb
import subprocess
import os
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.output_parsers import CommaSeparatedListOutputParser
from semantic_router.layer import RouteLayer
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain.docstore.document import Document
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter
from sentence_transformers import SentenceTransformer, util

#BERTscore
import bert_score
import logging
import transformers
transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)
from bert_score import BERTScorer
from BRAD import brad
from BRAD import rag

/home/machoi/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/machoi/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


## Single Document Calling

In [2]:
path = '/nfs/turbo/umms-indikar/shared/projects/RAG/databases/EXP2/'
vectordb, embeddings_model = brad.load_literature_db(persist_directory='/nfs/turbo/umms-indikar/shared/projects/RAG/databases/EXP2/')

Tue Jun 18 01:27:06 2024 INFO Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
/home/machoi/.local/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Tue Jun 18 01:27:08 2024 INFO Use pytorch device_name: cpu
Tue Jun 18 01:27:08 2024 INFO Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Tue Jun 18 01:27:09 2024 INFO Collection DB_cosine_cSize_700_cOver_200 is not created.


In [18]:
help(vectordb)

Help on Chroma in module langchain_chroma.vectorstores object:

class Chroma(langchain_core.vectorstores.VectorStore)
 |  Chroma(collection_name: 'str' = 'langchain', embedding_function: 'Optional[Embeddings]' = None, persist_directory: 'Optional[str]' = None, client_settings: 'Optional[chromadb.config.Settings]' = None, collection_metadata: 'Optional[Dict]' = None, client: 'Optional[chromadb.ClientAPI]' = None, relevance_score_fn: 'Optional[Callable[[float], float]]' = None, create_collection_if_not_exists: 'Optional[bool]' = True) -> 'None'
 |  
 |  `ChromaDB` vector store.
 |  
 |  To use, you should have the ``chromadb`` python package installed.
 |  
 |  Example:
 |      .. code-block:: python
 |  
 |              from langchain_chroma import Chroma
 |              from langchain_openai import OpenAIEmbeddings
 |  
 |              embeddings = OpenAIEmbeddings()
 |              vectorstore = Chroma("langchain_store", embeddings)
 |  
 |  Method resolution order:
 |      Chroma
 | 

In [5]:
test_list = list(vectordb.get()['ids'][:10])

In [6]:
source_title = []
path = "/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/"
print(path)
for metadata in vectordb.get()['metadatas']:
    source_title.append(metadata['source'].removeprefix(path).removesuffix(".pdf"))
no_repeat_source_title = set(source_title)
post_process_title = [x.lower() for x in no_repeat_source_title]
print(post_process_title)

/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/
['high-spatial-resolution multi-omics sequencing via deterministic barcoding in tissue', 'understanding transcriptional networks regulating initiation of cutaneous wound healing', 'transcriptional control of wound repair', 'optimal-transport analysis of single-cell gene expression identifies developmental trajectories in reprogramming', 'a predictive computational framework for direct reprogramming between human cell types', 'pederson - 2021 - a layperson encounter, on the “modified” rna world', 'long-term association of a transcription factor with its chromatin binding site can stabilize gene expression and cell fate commitment', 'a comprehensive library of human transcription', 'in vivo cellular reprogramming the next generation', 'transcription factors orchestrate dynamic interplay between genome topology and gene regulation during cell reprogramming', 'elite and stochastic models for induced pluripotent stem cell generation',

In [7]:
#Split into two methods?
def get_all_sources(prompt, path):
    prompt = prompt.lower()
    metadata_full = vectordb.get()['metadatas']
    source_list = [item['source'] for item in metadata_full]   
    real_source_list = [((item.replace(path, '')).removesuffix('.pdf')).lower() for item in source_list]
    db = pd.DataFrame({'id' :vectordb.get()['ids'] , 'metadatas' : real_source_list})
    filtered_df = db[db['metadatas'].apply(lambda x: x in prompt)]
    return real_source_list, filtered_df['id'].to_list()
        

In [8]:
prompt = 'Summarize the human transcription factors paper'
path = "/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/"
embeddings_model = HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5')
db_name = "new_DB_cosine_cSize_%d_cOver_%d" % (700, 200)
title_list, real_id_list = get_all_sources(prompt, path)
best_title, best_score = best_match(prompt, title_list)
title_list, real_id_list = get_all_sources(best_title, path)
text = vectordb.get(ids=real_id_list)['documents']
print(len(text))
#newdb = Chroma(persist_directory='/nfs/turbo/umms-indikar/shared/projects/RAG/databases/new_EXP3/', embedding_function=embeddings_model, collection_name=db_name)
#print(len(newdb.get()['documents']))
#newdb.add_texts(text)
#print(len(newdb.get()['documents']))

#things to do - hook this up with RAG (shouldnt be too bad)
#determine how to sift thru the prompt or go thru paper titles that are not well done
#ask about which paper db to use on turbo
#where to save this new db
#sometimes there is a weird error that occurs - maybe split up lower text stuff but idk how that effects runtime
#how to remove docs from database / make new one everytime since it keeps adding to it

Tue Jun 18 01:02:32 2024 INFO Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Tue Jun 18 01:02:32 2024 INFO Use pytorch device_name: cpu


NameError: name 'best_match' is not defined

In [ ]:
from sentence_transformers import SentenceTransformer, util


#Given the prompt, find the title and corresponding score that is the best match
def best_match(prompt, title_list):
    sentence_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
    unique_title_list = list(set(title_list))
    query_embedding = sentence_model.encode(prompt)
    passage_embedding = sentence_model.encode(unique_title_list)

    save_title = ""
    save_score = 0

    for score, title in zip(util.cos_sim(query_embedding, passage_embedding)[0], unique_title_list):
        if score > save_score:
            save_score = score
            save_title = title
    print(f"The best match is {save_title} with a score of {save_score}")
    return save_title, save_score

In [ ]:
save_title

In [ ]:
sentence_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
unique_title_list = list(set(title_list))
query_embedding = sentence_model.encode(prompt)
passage_embedding = sentence_model.encode(unique_title_list)

save_title = ""
save_score = 0

for score, title in zip(util.cos_sim(query_embedding, passage_embedding)[0], unique_title_list):
    if score > save_score:
        save_score = score
        save_title = title
print(f"The best match is {save_title} with a score of {save_score}")

In [ ]:
title_list, real_id_list = get_all_sources(save_title, "/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/")
text = vectordb.get(ids=real_id_list)['documents']
#print(text)

In [19]:


def restrictedDB(prompt, vectordb, path):
    #path = "/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/"
    embeddings_model = HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5')
    db_name = "new_DB_cosine_cSize_%d_cOver_%d" % (700, 200)
    title_list, real_id_list = get_all_sources(prompt, path)
    best_title, best_score = best_match(prompt, title_list)
    title_list, real_id_list = get_all_sources(best_title, path)
    text = vectordb.get(ids=real_id_list)['documents']
    newdb = Chroma(persist_directory='/nfs/turbo/umms-indikar/shared/projects/RAG/databases/new_EXP3/', embedding_function=embeddings_model, collection_name=db_name)
    newdb.add_texts(text)
    return best_score, newdb


#Given the prompt, find the title and corresponding score that is the best match
def best_match(prompt, title_list):
    sentence_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
    unique_title_list = list(set(title_list))
    query_embedding = sentence_model.encode(prompt)
    passage_embedding = sentence_model.encode(unique_title_list)

    save_title = ""
    #adjust the save_score to be the threshold cutoff - set to 0.75 but maybe thats too high
    save_score = 0.75

    for score, title in zip(util.cos_sim(query_embedding, passage_embedding)[0], unique_title_list):
        if score > save_score:
            save_score = score
            save_title = title
    print(f"The best match is {save_title} with a score of {save_score}")
    return save_title, save_score

#Split into two methods?
def get_all_sources(prompt, path):
    prompt = prompt.lower()
    metadata_full = vectordb.get()['metadatas']
    source_list = [item['source'] for item in metadata_full]   
    real_source_list = [((item.replace(path, '')).removesuffix('.pdf')).lower() for item in source_list]
    db = pd.DataFrame({'id' :vectordb.get()['ids'] , 'metadatas' : real_source_list})
    filtered_df = db[db['metadatas'].apply(lambda x: x in prompt)]
    return real_source_list, filtered_df['id'].to_list()

In [20]:
prompt = 'Summarize the human transcription factors paper'
path = "/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/"
#path = '/nfs/turbo/umms-indikar/shared/projects/RAG/databases/EXP2/'
vectordb, embeddings_model = brad.load_literature_db(persist_directory='/nfs/turbo/umms-indikar/shared/projects/RAG/databases/EXP2/')
best_score, text = restrictedDB(prompt, vectordb, path)
print(best_score)

Tue Jun 18 01:44:10 2024 INFO Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
/home/machoi/.local/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Tue Jun 18 01:44:11 2024 INFO Use pytorch device_name: cpu
Tue Jun 18 01:44:11 2024 INFO Collection DB_cosine_cSize_700_cOver_200 is not created.
Tue Jun 18 01:44:12 2024 INFO Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Tue Jun 18 01:44:13 2024 INFO Use pytorch device_name: cpu
Tue Jun 18 01:44:15 2024 INFO Load pretrained SentenceTransformer: multi-qa-MiniLM-L6-cos-v1
Tue Jun 18 01:44:16 2024 INFO Use pytorch device_name: cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

The best match is the human transcription factors with a score of 0.8029918670654297


Tue Jun 18 01:44:18 2024 INFO Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Tue Jun 18 01:44:18 2024 INFO Collection new_DB_cosine_cSize_700_cOver_200 is not created.


tensor(0.8030)


## PageRanker 

In [28]:
def remove_repeats(vectordb):
    df = pd.DataFrame({'id' :vectordb.get()['ids'] , 'documents' : vectordb.get()['documents']})
    repeated_ids = df[df.duplicated(subset='documents', keep='last')]['id'].tolist()
    vectordb.delete(repeated_ids)
    return vectordb

In [31]:
#vectordb.get()['documents']

In [48]:
#Given the prompt, find the title and corresponding score that is the best match
def adj_matrix_builder(prompt, vectordb):
    dimension = len(vectordb.get()['documents'])+1
    adj_matrix = np.zeros([dimension, dimension])
    
    sentence_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
    query_embedding = sentence_model.encode(prompt)
    passage_embedding = sentence_model.encode(vectordb.get()['documents'])
    pos = 1
    for score in zip(util.cos_sim(query_embedding, passage_embedding)[0]):
        adj_matrix[0][pos] = score[0]
        adj_matrix[pos][0] = score[0]
        pos += 1
    return adj_matrix

# Normalize columns of A
def normalize_adjacency_matrix(A):
    col_sums = A.sum(axis=0)
    return A / col_sums[np.newaxis, :]

#weighted pagerank algorithm
def pagerank_weighted(A, alpha=0.85, tol=1e-6, max_iter=100):
    n = A.shape[0]
    A_normalized = normalize_adjacency_matrix(A)
    v = np.ones(n) / n  # Initial PageRank vector

    for _ in range(max_iter):
        v_next = alpha * A_normalized.dot(v) + (1 - alpha) / n
        if np.linalg.norm(v_next - v, 1) < tol:
            break
        v = v_next

    return v

#reranker

def pagerank_rerank(prompt, vectordb):
    adj_matrix = adj_matrix_builder(prompt, vectordb)
    pagerank_scores = pagerank_weighted(A = adj_matrix)
    top_rank_scores = sorted(range(len(pagerank_scores)), key=lambda i: pagerank_scores[i], reverse=True)[1:11]
    df = pd.DataFrame({'id' :vectordb.get()['ids'] , 'documents' : vectordb.get()['documents'], 'metadatas' : vectordb.get()['metadatas']})
    reranked_df = df.iloc[top_rank_scores]
    return reranked_df

In [49]:
prompt = 'tell me about cell reprogramming'
rank_scores = pagerank_rerank(prompt, vectordb)
print(rank_scores)


Tue Jun 18 02:46:42 2024 INFO Load pretrained SentenceTransformer: multi-qa-MiniLM-L6-cos-v1
Tue Jun 18 02:46:43 2024 INFO Use pytorch device_name: cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/283 [00:00<?, ?it/s]

[4.59468655e-01 6.08485365e-05 7.72999648e-05 ... 5.36480820e-05
 4.53528530e-05 6.73818630e-05]


In [62]:
#first entry is always the prompt



{'page': 4, 'source': '/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/Transcription factories gene expression in unions.pdf'}
{'page': 17, 'source': '/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/Transcription Factories Genome Organization and Gene Regulation.pdf'}
{'page': 28, 'source': '/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/Updated Perspectives on Direct Vascular Cellular Reprogramming.pdf'}
{'page': 25, 'source': '/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/Optimal-Transport Analysis of Single-Cell Gene Expression Identifies Developmental Trajectories in Reprogramming.pdf'}
{'page': 17, 'source': '/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/Direct cell-fate conversion of somatic cells Toward regenerative medicine and industries.pdf'}
{'page': 7, 'source': '/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/In Vivo Cellular Reprogramming The Next Generation.pdf'}
{'page': 3, 'source': '/nfs/turbo/umms-indikar/shared/pr

In [53]:
#Given the prompt, find the title and corresponding score that is the best match
def adj_matrix_builder(prompt, vectordb):
    dimension = len(vectordb.get()['documents'])+1
    adj_matrix = np.zeros([dimension, dimension])
    
    sentence_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
    query_embedding = sentence_model.encode(prompt)
    passage_embedding = sentence_model.encode(vectordb.get()['documents'])
    pos = 1
    for score in zip(util.cos_sim(query_embedding, passage_embedding)[0]):
        adj_matrix[0][pos] = score[0]
        adj_matrix[pos][0] = score[0]
        pos += 1
    return adj_matrix

# Normalize columns of A
def normalize_adjacency_matrix(A):
    col_sums = A.sum(axis=0)
    return A / col_sums[np.newaxis, :]

#weighted pagerank algorithm
def pagerank_weighted(A, alpha=0.85, tol=1e-6, max_iter=100):
    n = A.shape[0]
    A_normalized = normalize_adjacency_matrix(A)
    v = np.ones(n) / n  # Initial PageRank vector

    for _ in range(max_iter):
        v_next = alpha * A_normalized.dot(v) + (1 - alpha) / n
        if np.linalg.norm(v_next - v, 1) < tol:
            break
        v = v_next

    return v

#reranker

def pagerank_rerank(prompt, vectordb):
    adj_matrix = adj_matrix_builder(prompt, vectordb)
    pagerank_scores = pagerank_weighted(A = adj_matrix)
    top_rank_scores = sorted(range(len(pagerank_scores)), key=lambda i: pagerank_scores[i], reverse=True)[1:11]
    df = pd.DataFrame({'id' :vectordb.get()['ids'] , 'documents' : vectordb.get()['documents'], 'metadatas' : vectordb.get()['metadatas']})
    reranked_df = df.iloc[top_rank_scores]
    return reranked_df

In [63]:
#I THINK THIS FRAMEWORK WORKS BETTER


#Given the prompt, find the title and corresponding score that is the best match
def adj_matrix_builder(docs, scores):
    dimension = len(docs)+1
    adj_matrix = np.zeros([dimension, dimension])
    pos = 1
    for score in scores:
        adj_matrix[0][pos] = score
        adj_matrix[pos][0] = score
        pos += 1
    return adj_matrix

# Normalize columns of A
def normalize_adjacency_matrix(A):
    col_sums = A.sum(axis=0)
    return A / col_sums[np.newaxis, :]

#weighted pagerank algorithm
def pagerank_weighted(A, alpha=0.85, tol=1e-6, max_iter=100):
    n = A.shape[0]
    A_normalized = normalize_adjacency_matrix(A)
    v = np.ones(n) / n  # Initial PageRank vector

    for _ in range(max_iter):
        v_next = alpha * A_normalized.dot(v) + (1 - alpha) / n
        if np.linalg.norm(v_next - v, 1) < tol:
            break
        v = v_next

    return v

#reranker

def pagerank_rerank(docs, scores):
    adj_matrix = adj_matrix_builder(docs, scores)
    pagerank_scores = pagerank_weighted(A = adj_matrix)
    top_rank_scores = sorted(range(len(pagerank_scores)), key=lambda i: pagerank_scores[i], reverse=True)[1:11]
    reranked_docs = [docs[i] for i in top_rank_scores]
    return reranked_docs

In [54]:
docs, scores

                                      id  \
0   0003eff7-c988-466f-97bd-85df69ca723a   
0   0003eff7-c988-466f-97bd-85df69ca723a   
0   0003eff7-c988-466f-97bd-85df69ca723a   
0   0003eff7-c988-466f-97bd-85df69ca723a   
0   0003eff7-c988-466f-97bd-85df69ca723a   
..                                   ...   
0   0003eff7-c988-466f-97bd-85df69ca723a   
0   0003eff7-c988-466f-97bd-85df69ca723a   
0   0003eff7-c988-466f-97bd-85df69ca723a   
0   0003eff7-c988-466f-97bd-85df69ca723a   
0   0003eff7-c988-466f-97bd-85df69ca723a   

                                            documents  
0   lOE(A)\nFIG.1.MapsoftheparentandMyoDretrovirus...  
0   lOE(A)\nFIG.1.MapsoftheparentandMyoDretrovirus...  
0   lOE(A)\nFIG.1.MapsoftheparentandMyoDretrovirus...  
0   lOE(A)\nFIG.1.MapsoftheparentandMyoDretrovirus...  
0   lOE(A)\nFIG.1.MapsoftheparentandMyoDretrovirus...  
..                                                ...  
0   lOE(A)\nFIG.1.MapsoftheparentandMyoDretrovirus...  
0   lOE(A)\nFIG.1.Mapso